In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
#grab the data from where it's stored
survey_responses = [
    "I can't figure out how to set up a payment plan.",
    "The system charged me twice for the same invoice!",
    "I’m trying to update my payment info, but it won’t let me.",
    "None of my clients received their invoices—what’s going on?",
    "Why do all the billing emails keep going to spam?",
    "The payment link is broken. My clients can’t pay!",
    "I’ve set up recurring payments, but they’re not processing.",
    "How do I resend an invoice? I can’t find the option.",
    "The payment plans are confusing. It’s not clear how they work.",
    "It says I’m overdue, but I set up a payment schedule!",
    "I tried paying, but the system says my card was declined. It wasn’t!",
    "Invoices aren’t showing as sent, but I emailed them days ago.",
    "All my payment reminders are being flagged as spam. Why?",
    "I’m trying to apply a discount, but it’s not showing on the invoice.",
    "The system doesn’t give me the option to send a receipt.",
    "My clients keep saying they didn’t get their invoices. This is embarrassing.",
    "Why are all my scheduled payments missing?",
    "I need to pause a payment plan, but there’s no option for that.",
    "The emails are going straight to my customers’ junk folders.",
    "I’m losing clients because they think we didn’t send invoices.",
    "Why isn’t there a clear option to set up automatic payments?",
    "I’m paying for premium, but I can’t even schedule a payment!",
    "None of my recurring invoices were sent this month.",
    "The system won’t let me edit a payment plan once it's set up.",
    "Why are the payment links expiring so quickly?",
    "It says my account is overdue, but I paid last week!",
    "I need to send a follow-up email, but the system keeps crashing.",
    "Is there a way to prevent my clients from being charged twice?",
    "I’m getting a lot of complaints about invoices going to spam.",
    "Why can’t I offer my clients partial payments?",
    "The system keeps charging my clients late fees even when they pay on time.",
    "Emails are going to the wrong addresses.",
    "I’ve double-checked, but my invoices aren’t showing as delivered.",
    "My customers keep missing the payment deadlines because of the spam issue.",
    "The payment reminders aren’t sending automatically.",
    "How can I stop sending overdue reminders to clients who’ve already paid?",
    "None of the payment options work for my international clients.",
    "Why doesn’t the system notify me when a payment fails?",
    "There’s no clear instruction on how to schedule payments.",
    "I tried updating my email templates, but the system won’t save the changes.",
    "I don’t get notified when payments come through.",
    "I have no idea how to offer different payment plans for different clients.",
    "The system is so confusing when it comes to payment plans.",
    "My clients keep asking why the invoices are going to spam.",
    "Can we get a way to preview invoices before sending?",
    "The payment confirmation email didn’t send. My client thinks I didn’t pay.",
    "There’s no way to split an invoice payment into two!",
    "It takes forever for payments to show up in the system.",
    "The system doesn’t allow me to send a thank you note after payment.",
    "Is there any way to send invoices to multiple recipients?",
    "I keep getting bounce-back emails from my clients.",
    "It says payment is overdue, but the client already paid via wire transfer.",
    "The system should allow for more flexible payment terms.",
    "My customers keep saying the invoice links don’t work.",
    "I’m trying to apply a credit to a customer’s account, but the system won’t let me.",
    "Why can’t I set up payment reminders to go out more frequently?",
    "Invoices are being flagged as suspicious in my clients' inboxes.",
    "The system doesn’t support partial payments at all!",
    "My payment was declined, but I’m sure there’s enough money in the account.",
    "My client’s email keeps bouncing back, but I have the correct email address.",
    "Invoices aren’t being sent out at the scheduled time.",
    "The system should notify customers before their payment plans expire.",
    "None of the payment plan emails are reaching my clients.",
    "I need to add a late fee manually, but there’s no option for that.",
    "There’s no way to edit a payment that’s been scheduled.",
    "The invoicing system should allow for custom payment schedules.",
    "Every invoice I send goes to my clients’ spam folders.",
    "Clients are getting frustrated because they can’t pay online.",
    "Why is it so hard to cancel a recurring payment?",
    "I can’t even find the button to stop automatic payments.",
    "I need to offer payment extensions, but the system won’t allow it.",
    "The system doesn’t track overdue payments correctly.",
    "I can’t resend a payment link once it’s expired.",
    "Customers are complaining about receiving multiple invoices for the same charge.",
    "Emails are being flagged as phishing. What’s going on?",
    "I sent an invoice weeks ago, but it shows as undelivered.",
    "The system won’t apply the late fees I’ve set up.",
    "None of the payment reminders are sending out.",
    "I need to offer a custom payment plan, but the system only has presets.",
    "Invoices aren’t being delivered, but there’s no error message.",
    "Why can’t I cancel a payment once it’s scheduled?",
    "I can’t figure out how to mark an invoice as paid.",
    "None of the automated payment reminders are working.",
    "My clients keep telling me they can’t find the invoices in their inbox.",
    "I tried to resend an invoice, but the system says it’s already sent.",
    "Emails are being flagged by spam filters way too often.",
    "Why does the system make it so hard to update payment details?",
    "It’s impossible to track which clients are on payment plans.",
    "I can’t apply credits to invoices after they’ve been sent.",
    "I need more control over the payment reminder schedule.",
    "The payment processing time is too long.",
    "It’s frustrating that my clients can’t pay in installments.",
    "Invoices are getting stuck in the system.",
    "I can’t figure out why my clients are being charged twice.",
    "There’s no way to send follow-up payment reminders.",
    "I need an easier way to edit payment plans after they’re set.",
    "All the emails are going straight to my clients’ spam folders.",
    "Why does the system keep flagging valid payment methods?",
    "Invoices aren’t syncing with our CRM.",
    "The email formatting is making the invoices look unprofessional.",
    "I tried to process a refund, but the system won’t let me.",
    "The invoice links expire too quickly for my clients to pay on time.",
    "I’m not getting notified when clients complete payments.",
    "Why can’t I set up multiple payment options for a single client?",
    "There’s no way to pause a recurring payment schedule.",
    "Invoices aren’t being sent out, but I don’t see any errors.",
    "The email attachments for invoices aren’t being delivered.",
    "Why are payment confirmations going to spam?",
    "I need to offer multiple payment terms for different clients.",
    "The invoice emails are showing up blank for my clients.",
    "Why isn’t there an option to email overdue payment notices manually?",
    "It’s hard to see which clients are on auto-pay.",
    "I can’t get the system to email receipts after payments.",
    "Why aren’t payment reminders being sent on time?",
    "Clients are missing payments because the invoices go to spam.",
    "I can’t manually adjust a payment once it’s been processed.",
    "The system should send alerts when invoices go unpaid.",
    "It’s hard to keep track of which clients are on payment plans.",
    "The invoices keep getting flagged as spam by my clients’ email systems.",
    "My clients are saying the payment page is confusing.",
    "There’s no clear way to cancel a scheduled payment.",
    "The system won’t allow me to edit invoices after they’re sent.",
    "None of the invoice links are working for my clients.",
    "The emails look too generic; clients aren’t recognizing them.",
    "I can’t figure out how to apply a late fee to a payment plan.",
    "There’s no notification when a payment plan fails.",
    "Clients keep saying they can’t find the invoice in their inbox.",
    "Why aren’t payments being processed in real-time?",
    "Invoices aren’t syncing with my payment gateway.",
    "I need more flexibility with the payment schedule options.",
    "This billing system is the worst I’ve ever used.",
    "It crashes all the time, so frustrating.",
    "I can’t figure out how to generate an invoice.",
    "Why is this so slow? It’s driving me crazy!",
    "The interface is cluttered and confusing.",
    "I hate the search function. It never works.",
    "I keep getting error messages for no reason.",
    "How do I even add a new client? Nothing is clear!",
    "It doesn’t save my work, I have to redo everything.",
    "Way too many bugs for a billing system.",
    "I’ve lost count of how many times it’s frozen today.",
    "The customer support is useless.",
    "I’m wasting too much time trying to pay my bill on the portal."
]

In [ ]:
# Convert to lowercase and remove punctuation
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text

In [ ]:
# Preprocess the survey responses
processed_responses = [preprocess_text(response) for response in survey_responses]

In [ ]:
# Create a document-term matrix
vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2)
doc_term_matrix = vectorizer.fit_transform(processed_responses)

In [ ]:
# Perform topic modeling using LDA
num_topics = 5
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_output = lda_model.fit_transform(doc_term_matrix)

In [ ]:
# Function to print top words for each topic
def print_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]
        print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")

In [ ]:
# Print the topics
print("Top words for each topic:")
print_topics(lda_model, vectorizer.get_feature_names_out(), 5)

In [ ]:
# Visualize topic distribution
plt.figure(figsize=(12, 6))
topic_proportions = lda_output.mean(axis=0)
plt.bar(range(num_topics), topic_proportions)
plt.title('Average Topic Distribution in Survey Responses')
plt.xlabel('Topic')
plt.ylabel('Proportion')
plt.xticks(range(num_topics), [f'Topic {i+1}' for i in range(num_topics)])
plt.savefig('topic_distribution.png')
plt.show()
plt.close()
print("\nTopic distribution chart saved as 'topic_distribution.png'")

In [ ]:
# Create a heatmap of topic distribution across responses
plt.figure(figsize=(12, 8))
sns.heatmap(lda_output, cmap='YlOrRd', cbar_kws={'label': 'Topic Probability'})
plt.title('Topic Distribution Across Survey Responses')
plt.xlabel('Topics')
plt.ylabel('Survey Responses')
plt.savefig('topic_heatmap.png')
plt.show()
plt.close()
print("Topic heatmap saved as 'topic_heatmap.png'")

In [ ]:
# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to analyze sentiment using VADER
def vader_sentiment(text):
    score = sia.polarity_scores(text)
    return score['compound']

# Analyze sentiment using VADER
vader_sentiments = [vader_sentiment(response) for response in survey_responses]

# Plot sentiment distribution using VADER
plt.figure(figsize=(12, 6))
plt.hist(vader_sentiments, bins=20, edgecolor='black')
plt.title('Sentiment Distribution of Survey Responses (VADER)')
plt.xlabel('Sentiment (Negative to Positive)')
plt.ylabel('Frequency')
plt.savefig('vader_sentiment_distribution.png')
plt.show()
plt.close()

print("VADER sentiment distribution chart saved as 'vader_sentiment_distribution.png'")

In [ ]:
#Word Cloud for Topic Visualization
# Create and save word clouds for each topic
def create_wordcloud(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        top_words = {feature_names[i]: topic[i] for i in topic.argsort()[:-num_top_words - 1:-1]}
        wordcloud = WordCloud(background_color='white').generate_from_frequencies(top_words)
        
        # Save word cloud as image
        plt.figure(figsize=(8, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Topic {topic_idx + 1} Word Cloud')
        plt.savefig(f'topic_{topic_idx+1}_wordcloud.png')
        plt.show()
        plt.close()

# Generate word clouds
create_wordcloud(lda_model, vectorizer.get_feature_names_out(), 10)

In [ ]:
#Coherence Score for Topic Model Validation
# Convert the processed responses into a Gensim dictionary format
texts = [response.split() for response in processed_responses]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Train the Gensim LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)

# Calculate coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()

print(f"\nLDA Coherence Score: {coherence_score:.2f}")